In [ ]:
import json
from pathlib import Path

import pandas as pd

DATA_DIR = Path("../data/russian_financial_news")

In [9]:
# 1. Чтение описаний тикеров
print("Загрузка описаний тикеров...")
with open(DATA_DIR / "tickers_description.json", encoding="utf-8") as f:
    tickers_description = json.load(f)

print(f"Загружено описаний для {len(tickers_description)} тикеров")
print(f"Примеры тикеров: {list(tickers_description.keys())[:5]}")

Загрузка описаний тикеров...
Загружено описаний для 238 тикеров
Примеры тикеров: ['ABIO', 'ABRD', 'AFKS', 'AFLT', 'AKRN']


In [10]:
# 2. Чтение коллекции новостей
print("Загрузка коллекции новостей...")
news_df = pd.read_parquet(DATA_DIR / "news_collection.parquet")
print(f"Загружено {len(news_df)} новостей")
print(f"\nКолонки: {news_df.columns.tolist()}")
print("\nПервые строки:")
news_df.head()

Загрузка коллекции новостей...
Загружено 91955 новостей

Колонки: ['title', 'body', 'date', 'time', 'tags', 'source']

Первые строки:


,title,body,date,time,tags,source
0,no title,"""После падения добычи в 2022-2023 годах Газпро...",2024-09-09,16:16:54,['цифры'],rdv
1,no title,""" Рейтинг акций комьюнити РДВ. #опрос """,2024-09-09,15:38:21,['опрос'],rdv
2,no title,"""Нефть Brent упала до $71, Urals - до $60 за б...",2024-09-09,14:40:17,['шокновости'],rdv
3,no title,"""️ Ставка ЦБ вряд ли уйдёт выше 18%. ЦБ переже...",2024-09-09,12:54:10,['VIPtalk'],rdv
4,no title,"""Эксперты клуба RDV PREMIUM 361° дали коммента...",2024-09-09,10:25:07,[],rdv


In [11]:
# 3. Чтение разметки от GPT4o
print("Загрузка разметки GPT4o...")
with open(DATA_DIR / "news_descriptions" / "news_descriptions_GPT4o.json", encoding="utf-8") as f:
    gpt4o_descriptions = json.load(f)

print(f"Загружено разметок: {len(gpt4o_descriptions)}")
print("\nПример разметки (первая запись):")
first_key = list(gpt4o_descriptions.keys())[0]
print(json.dumps(gpt4o_descriptions[first_key], ensure_ascii=False, indent=2))

Загрузка разметки GPT4o...
Загружено разметок: 15000

Пример разметки (первая запись):
{
  "thinking": "The article discusses Gazprom's production figures, highlighting a discrepancy between gas and oil revenue, and mentioning potential market changes in Russia's energy strategy. This is a financial report as it concerns company performance metrics (revenue figures) and an upcoming policy decision that could affect the gas market.",
  "article_type": "finance",
  "country": [
    "Russia"
  ],
  "sectors": [
    "Energy"
  ],
  "tickers": [
    "GAZP"
  ],
  "sentiment_score": -0.5
}


In [12]:
# 4. Чтение разметки от LLama3:8b
print("Загрузка разметки LLama3:8b...")
with open(DATA_DIR / "news_descriptions" / "news_description_LLama3_8b.json", encoding="utf-8") as f:
    llama_descriptions = json.load(f)

print(f"Загружено разметок: {len(llama_descriptions)}")
print("\nПример разметки (первая запись):")
first_key = list(llama_descriptions.keys())[0]
print(json.dumps(llama_descriptions[first_key], ensure_ascii=False, indent=2))

Загрузка разметки LLama3:8b...
Загружено разметок: 24398

Пример разметки (первая запись):
{
  "thinking": "The article discusses the recent trend in Gazprom's oil and gas production, highlighting that despite a significant increase in natural gas extraction, the revenue from gas sales has decreased compared to oil sales. This could potentially impact Russia's energy market and the country's overall economy.",
  "description": {
    "article_type": "finance",
    "country": [
      "Russia"
    ],
    "sectors": [
      "Energy"
    ],
    "tickers": [
      "GAZP"
    ],
    "sentiment_score": -0.5
  }
}


In [13]:
# 5. Чтение старой коллекции новостей (для соответствия индексации)
print("Загрузка старой коллекции новостей...")
news_old_df = pd.read_parquet(DATA_DIR / "news_descriptions" / "news_collection_old.parquet")
print(f"Загружено {len(news_old_df)} новостей")
print(f"\nКолонки: {news_old_df.columns.tolist()}")
news_old_df.head()

Загрузка старой коллекции новостей...
Загружено 79705 новостей

Колонки: ['Unnamed: 0', 'date', 'title', 'body', 'source']


,Unnamed: 0,date,title,body,source
0,0,2024-09-09 16:16:54,no title,"""После падения добычи в 2022-2023 годах Газпро...",rdv
1,1,2024-09-09 15:38:21,no title,""" Рейтинг акций комьюнити РДВ. #опрос """,rdv
2,2,2024-09-09 14:40:17,no title,"""Нефть Brent упала до $71, Urals - до $60 за б...",rdv
3,3,2024-09-09 12:54:10,no title,"""️ Ставка ЦБ вряд ли уйдёт выше 18%. ЦБ переже...",rdv
4,4,2024-09-09 10:25:07,no title,"""Эксперты клуба RDV PREMIUM 361° дали коммента...",rdv


In [14]:
# 6. Чтение данных свечей (candles) для разных периодов
def load_candles_data(period: str = "1d"):
    """
    Загружает данные свечей для указанного периода.

    Args:
        period: Период свечей ('10m', '1h', '1d')

    Returns:
        dict: Словарь с данными по каждому типу (open, high, low, close, volume, value)
    """
    candles_dir = DATA_DIR / "candles_data" / period

    if not candles_dir.exists():
        print(f"Директория {candles_dir} не найдена")
        return None

    candles_data = {}

    # Список файлов для загрузки
    files_to_load = ["open.csv", "high.csv", "low.csv", "close.csv"]
    if period in ["1d", "1h"]:
        files_to_load.append("volume.csv")
    if period == "10m":
        files_to_load.append("value.csv")

    for file_name in files_to_load:
        file_path = candles_dir / file_name
        if file_path.exists():
            df = pd.read_csv(file_path)
            candles_data[file_name.replace(".csv", "")] = df
            print(f"  Загружен {file_name}: {df.shape}")
        else:
            print(f"  Файл {file_name} не найден")

    return candles_data


# Загрузка данных для разных периодов
print("Загрузка данных свечей...")
candles_1d = load_candles_data("1d")
print("\nДанные для 1d загружены")
if candles_1d:
    print("Пример данных close для 1d:")
    if "close" in candles_1d:
        print(candles_1d["close"].head())

Загрузка данных свечей...
  Загружен open.csv: (553, 253)
  Загружен high.csv: (553, 253)
  Загружен low.csv: (553, 253)
  Загружен close.csv: (553, 253)
  Загружен volume.csv: (553, 253)

Данные для 1d загружены
Пример данных close для 1d:
        begin   ABIO   ABRD  ACKO    AFKS   AFLT   AGRO   AKRN   ALRS    AMEZ  \
0  2022-07-01  56.28  171.5   NaN  14.850  26.74  872.0  18660  66.24  21.220   
1  2022-07-04  56.90  173.0   NaN  15.000  27.12  854.4  18422  64.55  21.895   
2  2022-07-05  56.38  172.5   NaN  15.000  28.26  837.2  17902  64.99  21.500   
3  2022-07-06  60.68  172.0   NaN  15.163  27.34  843.8  18088  64.96  21.325   
4  2022-07-07  58.70  172.5   NaN  15.051  27.78  860.0  18402  65.35  21.375   

   ...  WUSH    YAKG  YDEX   YKEN   YKENP   YRSB  YRSBP  ZAYM  ZILL   ZVEZ  
0  ...   NaN  107.00   NaN  0.282  0.2785  170.0   82.5   NaN  2920  3.500  
1  ...   NaN  109.50   NaN  0.283  0.2785  172.0   83.0   NaN  3010  3.480  
2  ...   NaN  109.20   NaN  0.273  0.2680

In [15]:
# Загрузка данных для других периодов
candles_1h = load_candles_data("1h")
candles_10m = load_candles_data("10m")

  Загружен open.csv: (8039, 253)
  Загружен high.csv: (8039, 253)
  Загружен low.csv: (8039, 253)
  Загружен close.csv: (8039, 253)
  Загружен volume.csv: (8039, 253)
  Загружен open.csv: (44401, 248)
  Загружен high.csv: (44401, 248)
  Загружен low.csv: (44401, 248)
  Загружен close.csv: (44401, 248)
  Загружен value.csv: (44401, 248)


In [16]:
# 7. Функция для объединения новостей с разметкой
def merge_news_with_descriptions(news_df, descriptions_dict, model_name="GPT4o"):
    """
    Объединяет новости с их разметкой.

    Args:
        news_df: DataFrame с новостями
        descriptions_dict: Словарь с разметкой (ключи - индексы статей)
        model_name: Название модели для разметки

    Returns:
        DataFrame с объединенными данными
    """
    # Преобразуем словарь разметки в DataFrame
    descriptions_list = []
    for idx, desc in descriptions_dict.items():
        row = {"article_idx": int(idx)}
        # Извлекаем данные из разметки
        if isinstance(desc, dict):
            if "description" in desc:
                # Формат LLama3
                desc_data = desc["description"]
                row["thinking"] = desc.get("thinking", "")
            else:
                # Формат GPT4o
                desc_data = desc
                row["thinking"] = desc.get("thinking", "")

            row["article_type"] = desc_data.get("article_type", "")
            row["country"] = desc_data.get("country", [])
            row["sectors"] = desc_data.get("sectors", [])
            row["tickers"] = desc_data.get("tickers", [])
            row["sentiment_score"] = desc_data.get("sentiment_score", 0.0)
        else:
            row.update(
                {
                    "thinking": "",
                    "article_type": "",
                    "country": [],
                    "sectors": [],
                    "tickers": [],
                    "sentiment_score": 0.0,
                }
            )

        row["model"] = model_name
        descriptions_list.append(row)

    descriptions_df = pd.DataFrame(descriptions_list)

    # Объединяем с новостями по индексу
    # Предполагаем, что индекс в news_df соответствует article_idx
    merged_df = news_df.reset_index().merge(descriptions_df, left_on="index", right_on="article_idx", how="left")

    return merged_df


# Пример объединения с разметкой GPT4o
print("Объединение новостей с разметкой GPT4o...")
news_with_gpt4o = merge_news_with_descriptions(news_old_df, gpt4o_descriptions, "GPT4o")
print(f"Объединено {len(news_with_gpt4o)} записей")
news_with_gpt4o.head()

Объединение новостей с разметкой GPT4o...
Объединено 79705 записей


,index,Unnamed: 0,date,title,body,source,article_idx,thinking,article_type,country,sectors,tickers,sentiment_score,model
0,0,0,2024-09-09 16:16:54,no title,"""После падения добычи в 2022-2023 годах Газпро...",rdv,0.0,The article discusses Gazprom's production fig...,finance,[Russia],[Energy],[GAZP],-0.5,GPT4o
1,1,1,2024-09-09 15:38:21,no title,""" Рейтинг акций комьюнити РДВ. #опрос """,rdv,1.0,The article title suggests it is about a commu...,advertising,[],[],[],0.0,GPT4o
2,2,2,2024-09-09 14:40:17,no title,"""Нефть Brent упала до $71, Urals - до $60 за б...",rdv,2.0,The article discusses the decline in oil price...,finance,"[Russia, United States]","[Energy, Financial Service]",[],-0.5,GPT4o
3,3,3,2024-09-09 12:54:10,no title,"""️ Ставка ЦБ вряд ли уйдёт выше 18%. ЦБ переже...",rdv,3.0,The article discusses economic forecasts and f...,opinions,[Russia],"[Financial Service, Real Estate]",[MOEX],0.0,GPT4o
4,4,4,2024-09-09 10:25:07,no title,"""Эксперты клуба RDV PREMIUM 361° дали коммента...",rdv,4.0,The article is a commentary by RDV PREMIUM 361...,advertising,[Russia],[Real Estate],"[SMLT, ETLN, LSRG, PIKK]",0.5,GPT4o


In [17]:
# Объединение с разметкой LLama3
print("Объединение новостей с разметкой LLama3:8b...")
news_with_llama = merge_news_with_descriptions(news_old_df, llama_descriptions, "LLama3:8b")
print(f"Объединено {len(news_with_llama)} записей")
news_with_llama.head()

Объединение новостей с разметкой LLama3:8b...
Объединено 79705 записей


,index,Unnamed: 0,date,title,body,source,article_idx,thinking,article_type,country,sectors,tickers,sentiment_score,model
0,0,0,2024-09-09 16:16:54,no title,"""После падения добычи в 2022-2023 годах Газпро...",rdv,0.0,The article discusses the recent trend in Gazp...,finance,[Russia],[Energy],[GAZP],-0.5,LLama3:8b
1,1,1,2024-09-09 15:38:21,no title,""" Рейтинг акций комьюнити РДВ. #опрос """,rdv,1.0,The article appears to be about the community ...,opinions,[Russia],[],[],0.0,LLama3:8b
2,2,2,2024-09-09 14:40:17,no title,"""Нефть Brent упала до $71, Urals - до $60 за б...",rdv,2.0,The article discusses a significant decline in...,finance,"[Russia, USA]",[Energy],[$SBER],-1.0,LLama3:8b
3,3,3,2024-09-09 12:54:10,no title,"""️ Ставка ЦБ вряд ли уйдёт выше 18%. ЦБ переже...",rdv,3.0,The article discusses the potential future of ...,finance,[Russia],[Financial Service],[],0.0,LLama3:8b
4,4,4,2024-09-09 10:25:07,no title,"""Эксперты клуба RDV PREMIUM 361° дали коммента...",rdv,4.0,"Based on the article, it's clear that it falls...",finance,[Russia],"[Real Estate, Financial Service]","[SBER (Sberbank, not directly relevant but a m...",0.5,LLama3:8b


In [18]:
# 8. Сводная информация о датасете
print("=" * 60)
print("СВОДНАЯ ИНФОРМАЦИЯ О ДАТАСЕТЕ")
print("=" * 60)
print(f"\n1. Описания тикеров: {len(tickers_description)} записей")
print(f"2. Новости (основная коллекция): {len(news_df)} записей")
print(f"3. Новости (старая коллекция): {len(news_old_df)} записей")
print(f"4. Разметка GPT4o: {len(gpt4o_descriptions)} записей")
print(f"5. Разметка LLama3:8b: {len(llama_descriptions)} записей")
print("\n6. Данные свечей:")
if candles_1d and "close" in candles_1d:
    print(f"   - 1d: {len(candles_1d['close'])} записей")
if candles_1h and "close" in candles_1h:
    print(f"   - 1h: {len(candles_1h['close'])} записей")
if candles_10m and "close" in candles_10m:
    print(f"   - 10m: {len(candles_10m['close'])} записей")

СВОДНАЯ ИНФОРМАЦИЯ О ДАТАСЕТЕ

1. Описания тикеров: 238 записей
2. Новости (основная коллекция): 91955 записей
3. Новости (старая коллекция): 79705 записей
4. Разметка GPT4o: 15000 записей
5. Разметка LLama3:8b: 24398 записей

6. Данные свечей:
   - 1d: 553 записей
   - 1h: 8039 записей
   - 10m: 44401 записей
